In [ ]:
def calculate_facility_averages(df, metrics, levels_column="Level"):
    """
    Calculate the 50th percentile (median) for specified metrics at each Level.

    Parameters:
    - df: Pandas DataFrame containing test data.
    - metrics: List of metrics to calculate facility averages for (e.g., ["Max Bat Speed", "Average EV"]).
    - levels_column: Column name representing competition levels (default: "Level").

    Returns:
    - facility_averages: Dictionary containing median values for each metric grouped by level.
    """
    facility_averages = {}

    for metric in metrics:
        # Convert the metric column to numeric, coercing errors to NaN
        df[metric] = pd.to_numeric(df[metric], errors="coerce")

        # Group by level and calculate the median for the current metric
        facility_averages[metric] = (
            df.groupby(levels_column)[metric].median().dropna().to_dict()
        )

    return facility_averages

def generate_hitting_pr_board(df, athlete_id, pdf):
    """
    Generate a scoreboard-style PR Board for the athlete's hitting assessment.

    Parameters:
    - df: Pandas DataFrame containing test data.
    - athlete_id: Athlete's ID (string or integer).
    - pdf: PdfPages object for saving pages.
    """
    # Filter athlete-specific data
    athlete_data = df[df['ID'] == athlete_id]

    # Check for "Hitting" in Workout Type
    hitting_data = athlete_data[athlete_data['Workout Type'] == 'Hitting']
    if hitting_data.empty:
        print(f"Skipping Hitting PR Board for Athlete ID {athlete_id}: No hitting data available.")
        return  # Skip if no hitting data is available

    # Convert relevant metric columns to numeric (force invalid values to NaN)
    hitting_metrics = ["Max Bat Speed", "Max EV", "Max Distance", "Peak Hand Speed", "Average EV"]
    athlete_data[hitting_metrics] = athlete_data[hitting_metrics].apply(pd.to_numeric, errors="coerce")

    # Proceed with PR board generation
    athlete_data["Date"] = pd.to_datetime(athlete_data["Date"], errors="coerce")
    athlete_data = athlete_data.dropna(subset=["Date"]).sort_values(by="Date")

    # PR Calculation (handling NaN safely)
    prs = {}
    for metric in hitting_metrics:
        valid_data = athlete_data.dropna(subset=[metric])  # Ensure only valid numerical data is considered
        max_row = valid_data.loc[valid_data[metric].idxmax()] if not valid_data.empty else None
        prs[metric] = {
            "value": max_row[metric] if max_row is not None else None,
            "date": max_row["Date"].strftime("%m/%d/%Y") if max_row is not None else "N/A",
            "phase": max_row["Phase"] if max_row is not None else "N/A",
            "test_sub_type": max_row["Test Sub-Type"] if max_row is not None else "N/A",
        }

    # Load the custom font
    font_path = r"C:\Users\benoi\AppData\Local\Microsoft\Windows\Fonts\SfDigitalReadoutHeavyOblique-GKRA.ttf"
    custom_font = font_manager.FontProperties(fname=font_path)

    # Create the page layout
    fig, ax = plt.subplots(figsize=(14, 6))
    ax.axis("off")

    # PR Board Header
    header = f"{athlete_data['First Name'].iloc[0]} {athlete_data['Last Name'].iloc[0]} Hitting Assessment PR Board"
    ax.text(0.5, 0.95, header, fontsize=20, fontweight="bold", ha="center", va="center")

    # Display PR Metrics
    metrics_display_order = ["Max Bat Speed", "Max EV", "Max Distance", "Peak Hand Speed", "Average EV"]
    positions = [
        (0.15, 0.65),  # Max Bat Speed
        (0.5, 0.65),  # Max EV
        (0.85, 0.65),  # Max Distance
        (0.35, 0.3),  # Peak Hand Speed
        (0.65, 0.3),  # Average EV
    ]
    font_sizes = [84, 84, 84, 48, 48]  # Larger fonts for primary metrics

    for metric, pos, font_size in zip(metrics_display_order, positions, font_sizes):
        pr = prs[metric]
        value = pr["value"]
        value_text = f"{value:.2f}" if value is not None else "N/A"
        metric_text = f"{metric}"
        details = f"{pr['date']} | {pr['phase']} | {pr['test_sub_type']}"

        # Add metric name
        ax.text(pos[0], pos[1] + 0.12, metric_text, fontsize=16, ha="center", va="center", fontweight="bold")

        # Add value in digital scoreboard font
        ax.text(
            pos[0],
            pos[1],
            value_text,
            fontsize=font_size,
            fontproperties=custom_font,
            color="darkred",
            ha="center",
            va="center",
        )

        # Add smaller details below
        detail_offset = -0.1 if font_size == 84 else -0.06
        ax.text(pos[0], pos[1] + detail_offset, details, fontsize=8, ha="center", va="center", wrap=True)

    # Save the page to PDF
    pdf.savefig(fig)
    plt.close(fig)